In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# Model

In [ ]:
# Define the MLP model
class EEGClassifier(nn.Module):
    def __init__(self):
        super(EEGClassifier, self).__init__()
        self.fc1 = nn.Linear(20, 32)       # Input: 20 features → Hidden: 32 neurons
        self.fc2 = nn.Linear(32, 13)       # Hidden: 32 neurons → Output: 13 classes

    def forward(self, x):
        x = F.relu(self.fc1(x))            # Activation
        x = F.softmax(self.fc2(x), dim=1)  # Softmax for classification
        return x

# Instantiate the model
model = EEGClassifier()

# Optimizer and loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

